In [1]:
%matplotlib inline
import os
import importlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#airport geo
init_Data = data = pd.read_csv(
    os.path.join('data', 'train.csv.bz2')
)
airportname = init_Data['Departure'].unique()
airport = pd.read_csv("airport_data/airport-codes.txt",sep=",")
airport = airport[airport['local_code'].isin(airportname)]
airport = airport[airport['iso_country']=='US'].reset_index()
airport['StateCodes'] = airport['iso_region'].str.split('-',expand=True).iloc[:,1]
airport.rename(columns = {'municipality':'City'}, inplace=True)
airport.replace('Dallas-Fort Worth','Dallas', inplace=True)

In [3]:
#https://www.feiertagskalender.ch/ferien.php?geo=3537&jahr=2012&klasse=0&hl=en
#type of date:datetime.date
#need to turn datetime to date: df[''].dt.date
import datetime as dt
import holidays

#get holidays 
Holidays_US = holidays.US()[dt.date(2011,7, 1):dt.date(2013,6, 5)] + holidays.US()[dt.date(2012,1, 1):dt.date(2012,12, 31)]

school_break = pd.read_csv('schoolholiday.csv')
school_break.loc[:, 'start'] =pd.to_datetime(school_break.loc[:, 'start']).dt.date
school_break.loc[:, 'end'] = pd.to_datetime(school_break.loc[:, 'end']).dt.date

def nextworkday(date):
    one_day = dt.timedelta(days=1)
    next_day = date + one_day 
    while next_day.weekday() in holidays.WEEKEND or next_day in Holidays_US:
        next_day += one_day 
    return (next_day - date).days
    
def lastworkday(date):
    one_day = dt.timedelta(days=1)
    last_day = date - one_day 
    while last_day.weekday() in holidays.WEEKEND or last_day in Holidays_US:
        last_day -= one_day 
    return (date - last_day).days   

def schoolbreak(date):
    one_day = dt.timedelta(days=1)
    lastschoolday = lastworkday(date)
    nextschoolday = nextworkday(date)
    for i in range(len(school_break['start'])-1):
        if date >= school_break['start'][i] and date <= school_break['end'][i]:
            lastschoolday = (date - school_break['start'][i] + one_day).days
            nextschoolday = (school_break['end'][i] + one_day - date).days   
        elif date == school_break['end'][i] + one_day:
            lastschoolday = (date - school_break['start'][i] + one_day).days
        elif date == school_break['start'][i] - one_day:
            nextschoolday = (school_break['end'][i] + one_day - date).days   
    return lastschoolday, nextschoolday


In [4]:
#census, geo and eco data of state
#https://www.kaggle.com/lislejoem/us_energy_census_gdp_10-14
steco = pd.read_csv('state-eco.csv')
steco.set_index(['StateCodes'])
steco.drop([col for col in steco.columns if '2010' in col or '2014' in col], axis=1, inplace=True)
stgeo = steco[['StateCodes','State', 'Region', 'Division', 'Coast', 'Great Lakes']]
col = [col for col in steco.columns 
       if 'POP' in col 
       or 'RBIRTH' in col 
       or 'RDEATH' in col 
       or 'StateCodes' in col
       or 'RNETMIG' in col]
stcensus = steco[col].set_index(['StateCodes'])


In [5]:
# census, eco data of city
#https://apps.bea.gov/regional/histdata/releases/0615rpi/index.cfm
#https://apps.bea.gov/itable/iTable.cfm?ReqID=70&step=1#

city = airport['City'].unique()

citygdp = pd.read_csv('citygdp.csv')
citygdp = citygdp[citygdp['GeoName'].isin(city)].reset_index()
citygdp = citygdp[['GeoName',"GDP2011","GDP2012","GDP2013"]]
citygdp.rename(columns={"GeoName":'City'}, inplace=True)

cityincome = pd.read_csv('cityincome.csv')
cityincome = cityincome[cityincome['GeoName'].isin(city)].reset_index()
cityincome = cityincome[['GeoName',"RPI2011","RPI2012","RPI2013"]]
cityincome.rename(columns={"GeoName":'City'}, inplace=True)

citycensus = pd.read_csv('citycensus.csv')
citycensus = citycensus[citycensus['city'].isin(city)].reset_index()
citycensus = citycensus[['city',"2011","2012","2013"]]
citycensus.rename(
    columns={'city':'City',"2011":'POP2011',"2012":'POP2012',"2013":'POP2013'}, 
             inplace=True)

In [6]:
df_merged = pd.merge(
    airport, stgeo, how='left', on=['StateCodes'], sort=False
    )
df_merged = pd.merge(
    df_merged, stcensus, how='left', on=['StateCodes'], sort=False
    )
df_merged = pd.merge(
    df_merged, citygdp, how='left', on=['City'], sort=False
    )
df_merged = pd.merge(
    df_merged, citycensus, how='left', on=['City'], sort=False
    )
df_merged = pd.merge(
    df_merged, cityincome, how='left', on=['City'], sort=False
    )


In [7]:
df1 = df_merged.drop(
    [col for col in df_merged.columns if '2012' in col or '2013' in col], axis=1)
df2 = df_merged.drop(
    [col for col in df_merged.columns if '2011' in col or '2013' in col], axis=1)
df3 = df_merged.drop(
    [col for col in df_merged.columns if '2011' in col or '2012' in col], axis=1)

df1['Year']= 2011
df1.rename(columns =
           {'GDP2011':'GDP','POP2011':'POP','RPI2011':'RPI',
            'POPESTIMATE2011':'StPOP','RBIRTH2011':'StRBirth',
            'RDEATH2011':'StRDeath','RNETMIG2011':'StRMig'}, inplace=True)
df2['Year']= 2012
df2.rename(columns =
           {'GDP2012':'GDP','POP2012':'POP','RPI2012':'RPI',
           'POPESTIMATE2012':'StPOP','RBIRTH2012':'StRBirth',
            'RDEATH2012':'StRDeath','RNETMIG2012':'StRMig'}, inplace=True)
df3['Year']= 2013
df3.rename(columns =
           {'GDP2013':'GDP','POP2013':'POP','RPI2013':'RPI',
           'POPESTIMATE2013':'StPOP','RBIRTH2013':'StRBirth',
            'RDEATH2013':'StRDeath','RNETMIG2013':'StRMig'}, inplace=True)

df_merged = pd.concat([df1, df2, df3], axis=0)

In [8]:
df_merged.drop('continent', axis=1, inplace=True)

In [9]:
df_merged['Year']

0     2011
1     2011
2     2011
3     2011
4     2011
5     2011
6     2011
7     2011
8     2011
9     2011
10    2011
11    2011
12    2011
13    2011
14    2011
15    2011
16    2011
17    2011
18    2011
19    2011
0     2012
1     2012
2     2012
3     2012
4     2012
5     2012
6     2012
7     2012
8     2012
9     2012
10    2012
11    2012
12    2012
13    2012
14    2012
15    2012
16    2012
17    2012
18    2012
19    2012
0     2013
1     2013
2     2013
3     2013
4     2013
5     2013
6     2013
7     2013
8     2013
9     2013
10    2013
11    2013
12    2013
13    2013
14    2013
15    2013
16    2013
17    2013
18    2013
19    2013
Name: Year, dtype: int64

In [10]:
delay1= pd.read_csv("./airport_data/delay.csv")
delay1.dropna(axis=0, subset=['Facility'], how='any', inplace=True)
delay1['Date'] = pd.to_datetime(delay1['Date'])

In [11]:
delay2= pd.read_csv("airport_data/delays.csv")
delay2['Date'] = pd.to_datetime(delay2['Date'])

In [12]:
# city pair will add with departure and arrival
citypair =  pd.read_csv("airport_data/citypair.csv")
citypair['Date'] = pd.to_datetime(citypair['Date'])
citypair.dropna(axis=0, subset=['Date'], how='any', inplace=True)

In [13]:
air_total = pd.read_csv("airport_data/Total operation.csv", header=0)
air_total['Date'] = pd.to_datetime(air_total['Date'])
air_total = air_total[['Date','Facility','Tower_Operations', 'Total_Operations']]

In [14]:
air_operation = pd.read_csv("airport_data/Airport_operation.csv", header=1)
air_operation.dropna(axis=0, subset=['Facility'], how='any', inplace=True)
air_operation['Date'] = pd.to_datetime(air_operation['Date'])

In [15]:
airport_analysis = pd.read_csv("airport_data/Airport_analysis.csv")
airport_analysis['Date'] = pd.to_datetime(airport_analysis['Date'])

In [16]:
oil = pd.read_csv("airport_data/oil_series.csv")
oil['Date'] = pd.to_datetime(oil['Date'])
oil['oil_price'] = oil['Price']

In [17]:
df_m = delay1.merge(
    delay2, on=['Facility','Date'],how='left',sort='False').merge(
        air_total, on=['Facility','Date'],how='left',sort='False').merge(
            air_operation, on=['Facility','Date'],how='left',sort='False').merge(
                airport_analysis, on=['Facility','Date'],how='left',sort='False')

In [18]:
df_m = df_m.merge(oil, on='Date')
df_m.drop(axis=1, labels=['Unnamed: 0','Price'],inplace=True)

In [19]:
df_m['year'] = df_m['Date'].dt.year
df_m['month'] = df_m['Date'].dt.month
df_m['day'] = df_m['Date'].dt.day

In [20]:
df_m.rename(columns ={'Facility':'local_code','year':'Year'},inplace=True)

In [21]:
df_final = df_m.merge(df_merged, on=['local_code','Year'],how='left',sort=False)

In [22]:
df_final.to_csv('final_data.csv')

In [26]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14979 entries, 0 to 14978
Data columns (total 89 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Date                               14979 non-null  datetime64[ns]
 1   local_code                         14979 non-null  object        
 2   Total_ops                          14979 non-null  int64         
 3   Total Delays                       14979 non-null  int64         
 4   TMI_to                             14979 non-null  int64         
 5   Dep                                14979 non-null  int64         
 6   Abrn                               14979 non-null  int64         
 7   TMI_from_local                     14979 non-null  int64         
 8   TMI_from_non_local                 14979 non-null  int64         
 9   Total_occ_at                       14979 non-null  int64         
 10  Total_local_dep                   

In [27]:
citypair = pd.read_csv('airport_data/Citypair.csv')

In [28]:
citypair

,Departure,Arrival,Date,Flight Count,On-Time Gate Departure,On-Time Airport Departure,% On-Time Gate Arrivals,Arrivals With EDCT,Average EDCT,Gate Departure Delay,Taxi Out Delay,Average Taxi Out Time,Airport Departure Delay,Airborne Delay,Taxi In Delay,Block Delay,Gate Arrival Delay
0,ATL,BOS,01/01/2011,11.0,54.55,54.55,72.73,0.0,0.0,13.64,6.87,19.64,19.82,4.18,2.21,1.09,9.91
1,ATL,BOS,01/02/2011,15.0,66.67,53.33,86.67,0.0,0.0,10.33,6.37,18.53,14.93,3.33,1.28,0.00,4.40
2,ATL,BOS,01/03/2011,13.0,92.31,69.23,100.00,0.0,0.0,4.62,6.05,18.23,9.54,0.62,1.02,0.15,0.62
3,ATL,BOS,01/04/2011,15.0,80.00,60.00,86.67,0.0,0.0,6.13,7.38,20.27,12.20,2.80,2.15,0.53,3.20
4,ATL,BOS,01/05/2011,15.0,93.33,73.33,100.00,0.0,0.0,2.47,8.31,21.13,9.07,2.73,0.66,0.33,0.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398892,SFO,SEA,12/29/2013,21.0,95.24,90.48,95.24,0.0,0.0,3.00,2.88,15.00,4.90,2.33,3.55,1.57,2.81
398893,SFO,SEA,12/30/2013,22.0,90.91,86.36,90.91,0.0,0.0,6.77,3.27,15.23,8.23,7.14,3.57,2.82,6.73
398894,SFO,SEA,12/31/2013,19.0,73.68,73.68,84.21,0.0,0.0,10.53,3.55,15.79,12.58,5.95,2.68,2.58,9.32
398895,SFO,SEA,01/01/2014,21.0,80.95,76.19,85.71,0.0,0.0,8.29,3.63,15.67,10.38,2.67,2.42,0.52,7.86
